In [ ]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos

tests_data_path = os.path.join(os.path.abspath(''), os.path.pardir, "assets")

Create connection with speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50051)

Combine several speos file into one project feature

Here we are building a project with:
- An environment which is a road
- A blue car
- A red car

In [ ]:
from ansys.speos.workflow.combine_speos import combine_speos, LocatedSpeos
p = combine_speos(speos=speos, speos_to_combine=[LocatedSpeos(speos_file=os.path.join(tests_data_path, "Env_Simplified.speos", "Env_Simplified.speos"), axis_system=[0,0,0,1,0,0,0,1,0,0,0,1]),
                                                 LocatedSpeos(speos_file=os.path.join(tests_data_path, "BlueCar.speos", "BlueCar.speos"), axis_system=[2000, 0, 35000, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0]),
                                                 LocatedSpeos(speos_file=os.path.join(tests_data_path, "RedCar.speos", "RedCar.speos"), axis_system=[-4000, 0, 48000, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0])])
#print(p)


In [ ]:
import pyvista as pv
import numpy as np
import ansys.speos.core as core
from google.protobuf.internal.containers import RepeatedScalarFieldContainer

def __extract_part_mesh_info(part_data: core.Part, part_coordinate_info: RepeatedScalarFieldContainer = None) -> pv.PolyData:
        """
        extract mesh data info from a part.

        Parameters
        ----------
        part_data: ansys.api.speos.part.v1.part_pb2
            Part from scene.
        part_coordinate_info: RepeatedScalarFieldContainer
            message contains part coordinate info: origin, x_vector, y_vector, z_vector
        Returns
        -------
        pv.PolyData
            mesh data extracted.

        """

        def local2absolute(local_vertice: np.ndarray) -> np.ndarray:
            """
            convert local coordinate to global coordinate.

            Parameters
            ----------
            local_vertice: np.ndarray
                numpy array includes x, y, z info.

            Returns
            -------
            np.ndarray
             numpy array includes x, y, z info

            """
            global_origin = np.array(part_coordinate.origin)
            global_x = np.array(part_coordinate.x_vect) * local_vertice[0]
            global_y = np.array(part_coordinate.y_vect) * local_vertice[1]
            global_z = np.array(part_coordinate.z_vect) * local_vertice[2]
            return global_origin + global_x + global_y + global_z

        part_coordinate = core.AxisSystem()
        part_coordinate.origin = [0.0, 0.0, 0.0]
        part_coordinate.x_vect = [1.0, 0.0, 0.0]
        part_coordinate.y_vect = [0.0, 1.0, 0.0]
        part_coordinate.z_vect = [0.0, 0.0, 1.0]
        if part_coordinate_info is not None:
            part_coordinate.origin = part_coordinate_info[:3]
            part_coordinate.x_vect = part_coordinate_info[3:6]
            part_coordinate.y_vect = part_coordinate_info[6:9]
            part_coordinate.z_vect = part_coordinate_info[9:]
        part_mesh_info = None
        for body_idx, body_guid in enumerate(part_data.body_guids):
            body_item_data = speos.client.get_item(body_guid).get()
            for face_idx, face_guid in enumerate(body_item_data.face_guids):
                face_item_data = speos.client.get_item(face_guid).get()
                vertices = np.array(face_item_data.vertices)
                facets = np.array(face_item_data.facets)
                vertices = vertices.reshape(-1, 3)
                vertices = np.array([local2absolute(vertice) for vertice in vertices])
                facets = facets.reshape(-1, 3)
                temp = np.full(facets.shape[0], 3)
                temp = np.vstack(temp)
                facets = np.hstack((temp, facets))
                face_mesh_data = pv.PolyData(vertices, facets)
                if part_mesh_info is None:
                    part_mesh_info = face_mesh_data
                else:
                    part_mesh_info = part_mesh_info.append_polydata(face_mesh_data)
        return part_mesh_info

def preview(project: script.Project) -> None:
    """preview cad bodies inside the scene."""
    _preview_mesh = pv.PolyData()
    
    root_part_data = speos.client.get_item(project.scene_link.get().part_guid).get()
    if len(root_part_data.parts) != 0:
        for part_idx, part_item in enumerate(root_part_data.parts):
            part_item_data = speos.client.get_item(part_item.part_guid).get()
            poly_data = __extract_part_mesh_info(part_item_data, part_item.axis_system)
            if poly_data is not None:
                _preview_mesh = _preview_mesh.append_polydata(poly_data)

    poly_data = __extract_part_mesh_info(root_part_data)
    if poly_data is not None:
        _preview_mesh = _preview_mesh.append_polydata(poly_data)

    p = pv.Plotter()
    p.add_mesh(_preview_mesh, show_edges=True)
    p.show()

In [ ]:
preview(project=p)

Complete the project with source/sensor/simulation

We are adding a camera sensor to have output results, a luminaire to have a light source.  
And we gather the sensor and the camera into a simulation. (we will compute it just after)

In [ ]:
ssr = p.create_sensor(name="Camera.1")
ssr.set_camera().set_distortion_file_uri(uri=os.path.join(tests_data_path, "CameraInputFiles", "CameraDistortion_190deg.OPTDistortion"))\
                .set_mode_photometric().set_transmittance_file_uri(uri=os.path.join(tests_data_path, "CameraInputFiles", "CameraTransmittance.spectrum"))\
                .set_mode_color().set_red_spectrum_file_uri(uri=os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityRed.spectrum"))\
                                 .set_blue_spectrum_file_uri(uri=os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityBlue.spectrum"))\
                                 .set_green_spectrum_file_uri(uri=os.path.join(tests_data_path, "CameraInputFiles", "CameraSensitivityGreen.spectrum"))
ssr.set_camera().set_axis_system([-2000, 1500, 11000, -1, 0, 0, 0, 1, 0, 0, 0, -1])
ssr.commit()

src = p.create_source(name="Luminaire.1")
src.set_luminaire().set_intensity_file_uri(uri=os.path.join(tests_data_path, "IES_C_DETECTOR.ies")).set_spectrum().set_daylightfluorescent()
src.set_luminaire().set_axis_system([0, 10000, 50000, 1, 0, 0, 0, 1, 0, 0, 0, 1])
src.commit()

sim = p.create_simulation(name="Inverse.1")
sim.set_inverse()
sim.set_sensor_paths(["Camera.1"]).set_source_paths(["Luminaire.1"])
sim.commit()

Compute

In [ ]:
sim.compute_CPU()

Watch results

In [ ]:
from ansys.speos.workflow.open_result import open_result_image

open_result_image(simulation_feature=sim, result_name="Camera.1.png")

Modify geometry - compute - watch result

Here we are moving the blue car. The new position is clother to the camera sensor.

In [ ]:
blue_car_sub_part = p.find(name="RootPart/BlueCar")
blue_car_sub_part.set_axis_system([2000,0.0,20000,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0])
blue_car_sub_part.commit()

sim.compute_CPU()

open_result_image(simulation_feature=sim, result_name="Camera.1.png")

Modify camera - compute - watch result

He we are modifying camera characteristic: focal length.

In [ ]:
cam1 = p.find(name="Camera.1")
cam1.set_camera().set_focal_length(value=10)
cam1.commit()

sim.compute_CPU()

open_result_image(simulation_feature=sim, result_name="Camera.1.png")